# This ignores test data

In [35]:
# Based on example at
# https://github.com/biplav-s/course-nl/blob/master/l9-ml-review/Classification%20-%20Fake%20news.ipynb

In [36]:
import pandas as pd

In [37]:
df = pd.read_csv('../data/Constraint_English_Train.csv')
df.head()

,id,tweet,label
0,1,The CDC currently reports 99031 deaths. In gen...,real
1,2,States reported 1121 deaths a small rise from ...,real
2,3,Politically Correct Woman (Almost) Uses Pandem...,fake
3,4,#IndiaFightsCorona: We have 1524 #COVID testin...,real
4,5,Populous states can generate large case counts...,real


In [38]:
nRow, nCol = df.shape
print(f'INFO: There are {nRow} rows and {nCol} columns in the training set.')

INFO: There are 6420 rows and 3 columns in the training set.


In [39]:
# Clean of white spaces
df = df.applymap(lambda x: x.strip() if isinstance(x, str) else x)
# Print statistics
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6420 entries, 0 to 6419
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      6420 non-null   int64 
 1   tweet   6420 non-null   object
 2   label   6420 non-null   object
dtypes: int64(1), object(2)
memory usage: 150.6+ KB


In [40]:
# We will have vector representation before we can do classification
# Do imports
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

In [41]:
# We will consider 1- and 2- gram
transformer = TfidfTransformer(smooth_idf=False)
count_vectorizer = CountVectorizer(ngram_range=(1, 2))

In [42]:
# fit training data to the count vectorizer
train_counts = count_vectorizer.fit_transform(df['tweet'].values)

#fit the ngrams count to the tfidf transformers
train_tfidf = transformer.fit_transform(train_counts)

In [43]:
# We can extract the label column from df dataframe to be the target 'y' variable
targets = df['label'].values

In [44]:
from sklearn.model_selection import train_test_split

#  If train-test size is not initialized, test_size will be set to 0.25 and train_set = 1-test_size
X_train, X_test, y_train, y_test = train_test_split(train_tfidf, targets, random_state=0)

In [45]:
# Import for prediction - we will use two methods
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix

In [46]:
# Try titles on random forest
RandomFC= RandomForestClassifier(n_estimators=5)
RandomFC.fit(X_train, y_train)

RandomForestClassifier(n_estimators=5)

In [47]:
# Print accuracy
print('Accuracy of randomforest classifier on training set: {:.2f}'.format(RandomFC.score(X_train, y_train)))
print('Accuracy of randomforest classifier on test set: {:.2f}'.format(RandomFC.score(X_test, y_test)))
CM = confusion_matrix(y_test, RandomFC.predict(X_test))
print(CM)

Accuracy of randomforest classifier on training set: 0.99
Accuracy of randomforest classifier on test set: 0.87
[[685  87]
 [115 718]]


In [48]:
# Also try logistic regression
logreg = LogisticRegression(C=1e5)
logreg.fit(X_train, y_train)

LogisticRegression(C=100000.0)

In [49]:
# Print accuracy
print('Accuracy of Logreg classifier on training set: {:.2f}'.format(logreg.score(X_train, y_train)))
print('Accuracy of Logreg classifier on test set: {:.2f}'.format(logreg.score(X_test, y_test)))
CM = confusion_matrix(y_test, logreg.predict(X_test))
print(CM)

Accuracy of Logreg classifier on training set: 1.00
Accuracy of Logreg classifier on test set: 0.93
[[719  53]
 [ 54 779]]


In [50]:
print ("Learned model's information - ")
print ('Classes - ' + str(logreg.classes_))
print ('Intercept - ' + str(logreg.intercept_))
print ('Coefficients - ' + str(logreg.coef_))

Learned model's information - 
Classes - ['fake' 'real']
Intercept - [-6.70909457]
Coefficients - [[0.36933017 0.04664991 1.42104141 ... 0.         0.         0.        ]]


# We will now do properly on test and test data